## 1. mport libraries

In [ ]:
# Import libraries
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ML libraries
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report, roc_curve, auc
)
from sklearn.feature_selection import mutual_info_regression
from xgboost import XGBClassifier

# Imbalance handling
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

# Styling
sns.reset_defaults()
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid", palette="crest")

print("All libraries imported successfully")

✓ All libraries imported successfully


<a id="loading"></a>

---

## 2. Data Loading, Initial Exploration & Cleaning Data

In [3]:
# Load datasets
app_data = pd.read_csv('application_record.csv')
credit_data = pd.read_csv('credit_record.csv')

print(f"Data Record Information \n")
print(f"Application Records: {app_data.shape[0]:,} rows, {app_data.shape[1]} columns")
print(f"Credit Records: {credit_data.shape[0]:,} rows, {credit_data.shape[1]} columns")
print(f"Unique Applicants: {app_data['ID'].nunique():,}")
print(f"Common IDs: {len(set(app_data['ID']).intersection(set(credit_data['ID']))):,}")

Data Record Information 

Application Records: 438,557 rows, 18 columns
Credit Records: 1,048,575 rows, 3 columns
Unique Applicants: 438,510
Common IDs: 36,457


In [4]:
print("\n" + "="*30)
print("APPLICATION DATA SAMPLE")
print("="*30)
display(app_data.head())

print("\n" + "="*30)
print("CREDIT RECORD SAMPLE")
print("="*30)
display(credit_data.head(20))


APPLICATION DATA SAMPLE


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0



CREDIT RECORD SAMPLE


,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
5,5001712,-1,C
6,5001712,-2,C
7,5001712,-3,C
8,5001712,-4,C
9,5001712,-5,C


#### Size of Data

In [5]:
print("="*30)
print("DATA INFO")
print("="*30)

print("\nCredit Data Info:\n")
credit_data.info()

print("\n" + "="*55)

print("\nApplication Data Info:\n")
app_data.info()

print("\n" + "="*55)

print("\n\n" + "="*30)
print("Missing Values")
print("="*30)
print("\nCredit Data:\n")
print(credit_data.isnull().sum())
print("\n" + "="*55)
print("\nApplication Data:\n")
print(app_data.isnull().sum())

DATA INFO

Credit Data Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   ID              1048575 non-null  int64 
 1   MONTHS_BALANCE  1048575 non-null  int64 
 2   STATUS          1048575 non-null  object
dtypes: int64(2), object(1)
memory usage: 24.0+ MB


Application Data Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438557 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   438557 non-null  int64  
 1   CODE_GENDER          438557 non-null  object 
 2   FLAG_OWN_CAR         438557 non-null  object 
 3   FLAG_OWN_REALTY      438557 non-null  object 
 4   CNT_CHILDREN         438557 non-null  int64  
 5   AMT_INCOME_TOTAL     438557 non-null  float64
 6   NAME_INCOME_TYPE     438557

#### Handles Null values in OCCUPATION_TYPE

In [6]:
#Set OCCUPATION_TYPE as unemployed for people who have positive values in DAYS_EMPLOYED and unknown for the rest blank value

app_data['OCCUPATION_TYPE'] = np.where(
    app_data['OCCUPATION_TYPE'].isna() | (app_data['OCCUPATION_TYPE'] == ''),
    np.where(app_data['DAYS_EMPLOYED'] > 0, 'Unemployed', 'Unknown'),
    app_data['OCCUPATION_TYPE']
)

app_data

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,Unknown,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,Unknown,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,M,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,Unemployed,1.0
438553,6840222,F,N,N,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1,0,0,0,Laborers,1.0
438554,6841878,F,N,N,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1.0
438555,6842765,F,N,Y,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,1,0,0,0,Unemployed,2.0


In [7]:
print("\n" + "="*30)
print("Missing Values")
print("="*30)
print("\nApplication Data:\n")
print(app_data.isnull().sum())


Missing Values

Application Data:

ID                     0
CODE_GENDER            0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
CNT_CHILDREN           0
AMT_INCOME_TOTAL       0
NAME_INCOME_TYPE       0
NAME_EDUCATION_TYPE    0
NAME_FAMILY_STATUS     0
NAME_HOUSING_TYPE      0
DAYS_BIRTH             0
DAYS_EMPLOYED          0
FLAG_MOBIL             0
FLAG_WORK_PHONE        0
FLAG_PHONE             0
FLAG_EMAIL             0
OCCUPATION_TYPE        0
CNT_FAM_MEMBERS        0
dtype: int64


#### Add additional columns for cleaning data

In [8]:
# Convert DAYS_BIRTH to age in years
app_data['AGE'] = (-app_data['DAYS_BIRTH'] / 365.25).astype(int)

# Convert DAYS_EMPLOYED to years employed (handle positive values as 0)
app_data['YEARS_EMPLOYED'] = np.where(
    app_data['DAYS_EMPLOYED'] > 0, 
    0, 
    (-app_data['DAYS_EMPLOYED'] / 365.25).astype(int)
)


# Family size vs children ratio
app_data['CHILDREN_RATIO'] = app_data['CNT_CHILDREN'] / app_data['CNT_FAM_MEMBERS']

In [9]:
# Convert Y/N columns to 1/0
binary_cols = ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY']

for col in binary_cols:
    app_data[col] = app_data[col].map({'Y': 1, 'N': 0})

In [10]:
app_data

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,AGE,YEARS_EMPLOYED,INCOME_GROUP,CHILDREN_RATIO
0,5008804,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,Unknown,2.0,32,12,Very High,0.0
1,5008805,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,Unknown,2.0,32,12,Very High,0.0
2,5008806,M,1,1,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,58,3,Medium,0.0
3,5008808,F,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,52,8,High,0.0
4,5008809,F,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,52,8,High,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,M,0,1,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,Unemployed,1.0,62,0,Medium,0.0
438553,6840222,F,0,0,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1,0,0,0,Laborers,1.0,43,8,Medium,0.0
438554,6841878,F,0,0,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1.0,22,1,Low,0.0
438555,6842765,F,0,1,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,1,0,0,0,Unemployed,2.0,59,0,Low,0.0


In [11]:
#Export file
app_data.to_csv('application_record_Cleaned.csv')

### Add column for credit_record

In [12]:
credit_data['Label'] = (
    credit_data.groupby('ID')['STATUS']
    .transform(lambda x: 'Bad' if any(s in ['1','2','3','4','5'] for s in x) else 'Good')
)

In [13]:
credit_data

,ID,MONTHS_BALANCE,STATUS,Label
0,5001711,0,X,Good
1,5001711,-1,0,Good
2,5001711,-2,0,Good
3,5001711,-3,0,Good
4,5001712,0,C,Good
...,...,...,...,...
1048570,5150487,-25,C,Good
1048571,5150487,-26,C,Good
1048572,5150487,-27,C,Good
1048573,5150487,-28,C,Good


### Merge Datasets

In [14]:
final_data = app_data.merge(credit_data, on='ID', how='inner')

In [15]:
final_data

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,AGE,YEARS_EMPLOYED,INCOME_GROUP,CHILDREN_RATIO,MONTHS_BALANCE,STATUS,Label
0,5008804,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,Unknown,2.0,32,12,Very High,0.0,0,C,Bad
1,5008804,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,Unknown,2.0,32,12,Very High,0.0,-1,C,Bad
2,5008804,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,Unknown,2.0,32,12,Very High,0.0,-2,C,Bad
3,5008804,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,Unknown,2.0,32,12,Very High,0.0,-3,C,Bad
4,5008804,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,Unknown,2.0,32,12,Very High,0.0,-4,C,Bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777710,5150337,M,0,1,0,112500.0,Working,Secondary / secondary special,Single / not married,Rented apartment,-9188,-1193,1,0,0,0,Laborers,1.0,25,3,Medium,0.0,-9,0,Bad
777711,5150337,M,0,1,0,112500.0,Working,Secondary / secondary special,Single / not married,Rented apartment,-9188,-1193,1,0,0,0,Laborers,1.0,25,3,Medium,0.0,-10,2,Bad
777712,5150337,M,0,1,0,112500.0,Working,Secondary / secondary special,Single / not married,Rented apartment,-9188,-1193,1,0,0,0,Laborers,1.0,25,3,Medium,0.0,-11,1,Bad
777713,5150337,M,0,1,0,112500.0,Working,Secondary / secondary special,Single / not married,Rented apartment,-9188,-1193,1,0,0,0,Laborers,1.0,25,3,Medium,0.0,-12,0,Bad


In [17]:
final_data.to_csv('MergeData.csv')